## WMG Analysis - Spotify Analysis POC

### useful links
https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6
https://rareloot.medium.com/extracting-spotify-data-on-your-favourite-artist-via-python-d58bc92a4330


In [7]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotify_credentials as cred

In [9]:
client_credentials_manager = SpotifyClientCredentials(client_id=cred.client_id, client_secret=cred.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

name = "{Tom Misch}" #chosen artist
result = sp.search(name) #search query
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uiEZYehlNivdK3iQyAbye'},
  'href': 'https://api.spotify.com/v1/artists/1uiEZYehlNivdK3iQyAbye',
  'id': '1uiEZYehlNivdK3iQyAbye',
  'name': 'Tom Misch',
  'type': 'artist',
  'uri': 'spotify:artist:1uiEZYehlNivdK3iQyAbye'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/1Z8ODXyhEBi3WynYw0Rya6'},
  'href': 'https://api.spotify.com/v1/artists/1Z8ODXyhEBi3WynYw0Rya6',
  'id': '1Z8ODXyhEBi3WynYw0Rya6',
  'name': 'De La Soul',
  'type': 'artist',
  'uri': 'spotify:artist:1Z8ODXyhEBi3WynYw0Rya6'}]

In [11]:
#Extract Artist's uri
artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
#Pull all of the artist's albums
sp_albums = sp.artist_albums(artist_uri, album_type='album')
#Store artist's albums' names' and uris in separate lists
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
    
print(album_names)
print(album_uris)
#Keep names and uris in same order to keep track of duplicate albums

['What Kinda Music', 'Geography', 'Geography', 'Beat Tape 2', 'Beat Tape 1', 'Beat Tape 1']
['spotify:album:6iOCv7oGL5sGi2aVnRz2BI', 'spotify:album:2g49Kw03cpc5HQygEw450P', 'spotify:album:28enuddLPEA914scE6Drvk', 'spotify:album:5IW5ko3B1W5doRD3YH9DV8', 'spotify:album:2FVfODkF1inBzSwB2plsYk', 'spotify:album:6wHutK9fqU7pmLkoNCr7zN']


In [12]:
def albumSongs(uri):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album'].append(album_names[album_count]) #append album name tracked via album_count
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [13]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    albumSongs(i)
    print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

Album What Kinda Music songs has been added to spotify_albums dictionary
Album Geography songs has been added to spotify_albums dictionary
Album Geography songs has been added to spotify_albums dictionary
Album Beat Tape 2 songs has been added to spotify_albums dictionary
Album Beat Tape 1 songs has been added to spotify_albums dictionary
Album Beat Tape 1 songs has been added to spotify_albums dictionary


In [14]:
def audio_features(album):
    
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [15]:
import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 playlists completed
Loop #: 5
Elapsed Time: 15.174092054367065 seconds


In [16]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

90

In [17]:
df = pd.DataFrame.from_dict(dic_df)

In [18]:
print(len(df))
final_df = df.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
print(len(final_df))

90
62


In [21]:
df.head(100)
final_df.to_csv("tomMisch.csv", index=False, encoding='utf8')